# Wstępna analiza IUM
Rafał Bosko <br>

In [23]:
import jsonlines
import pandas as ps

## Problem biznesowy
<blockquote>
Predykcja czy dana sesja użytkownika zakończy się zakupem na podstawie danych sesji. Skuteczna klasyfikacja sesji, które zakończą się zakupem umożliwi konsultantom szybsze rozwiązywanie problemów
</blockquote>

## Biznesowe kryterium sukcesu
System powinien przewidywać sesje, które zakończą się zakupem z efektywnością porównywalną z innymi istniejącymi rozwiązaniami wymienionymi w [literaturze](#literatura).

## Zadanie modelowania
Zadaniem jest klasyfikacja binarna odpowiadająca na pytanie "Czy sesja użytkownika zakończy się zakupem?".
Do dyspozycji mamy historyczne dane składające się z:
* bazy użytkowników
* katalogu produktów
* historia sesji
* dane dotyczące wysyłki zakupionych produktów

## Analityczne kryterium sukcesu
Uzyskanie odpowiednio wysokiej skuteczności predykcji mierzonej poprzez *Accuracy* obliczanej jako: 
$\frac{TP + TN}{TP + TN + FP + FN}$

Rozważaliśmy również *F1 score*, obliczanej jako: $2 * \frac{Precision * Recall}{Precision + Recall}$ gdzie: <br>
$Precision = \frac{TP}{TP + FP}$ oraz $Recall = \frac{TP}{TP + FN}$

Nie zdecydowaliśmy jednak wstępnie na tą miarę, ponieważ:
* jest trudniejsza do interpretacji
* F1 sprawdza się lepiej od Accuraccy w przypadku, gdy jest duża dysproporcja między klasami, a w naszym przypadku takiego problemu nie ma

## Analiza zbioru danych
Wstępna analiza danych pozwoliła nam odrzucić dane o przesyłkach, ponieważ te dane są dostarczane po zakupie i nie wpływają na zakup
* *czy aby na pewno*

Informacje zawarte w pozostałych tabelach są dostępne w trakcie trwania sesji, więc przeszliśmy do bardziej szczegółowej analizy tego fragmentu danych.

Postanowiliśmy wyróżnić ze zbioru danych następujące informacje:
* [*user_id*](#userId) - identyfikator użytkownika
* [*purchase*](#purchase) - wartość boolowska odpowiadająca na pytanie czy sesja zakończyła się zakupem
* [*offered_discount*](#discount) - wysokość zniżki
* [*duration*](#duration) - długość trwania sesji w sekundach
* [*click_rate*](#click_rate) - liczba zdarzeń(event'ów) do aktualnego rekordu na minutę
* [*item_views*](#item_views) - liczba obejrzanych przedmiotów
* [*unique_item_views*](#unique_item_views) - liczba przejrzanych unikalnych przedmiotów
* [*unique_categories*](#unique_categories) - liczba przejrzanych unikalnych kategorii

## Brakujące wartości
* user_id - w miarę możliwości uzupełnialiśmy wartością user_id, która występowała najczęściej w danej sesji. Ten sposób pozwolił na ~16-krotne zmniejszenie brakujących wartości w tej kolumnie. Pozostałe wartości zamienione były na zera, które oznaczają brak przypisanego użytkownika.
* unique_categories - przy wyliczaniu tego atrybutu traktowaliśmy brakujące wartości w *category_path* jako osobną kategorię

Powyższe informacje zapisaliśmy w pliku *merged_dataset*

In [25]:
sessions = pd.read_csv("../data/processed/merged_dataset", sep=' ')
sessions.head()

,session_id,user_id,offered_discount,price,duration,weekend,weekday,hour,unique_item_views,item_views,click_rate,unique_categories,purchase,last_session_purchase
0,100048,102,0,553.00,0.0,False,4.0,4.0,1.0,1.0,0.000000,1.0,True,False
1,100048,102,0,553.00,94.0,False,4.0,4.0,1.0,2.0,1.276596,1.0,True,False
2,100081,102,15,553.00,0.0,False,4.0,22.0,1.0,1.0,0.000000,1.0,True,True
3,100081,102,15,553.00,146.0,False,4.0,22.0,1.0,2.0,0.821918,1.0,True,True
4,100026,102,15,99.99,0.0,True,5.0,8.0,1.0,1.0,0.000000,1.0,True,True
